In [1]:
# ドライブをマウント
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
!apt install aptitude swig

Reading package lists... Done
Building dependency tree       
Reading state information... Done
aptitude is already the newest version (0.8.10-6ubuntu1).
swig is already the newest version (3.0.12-1).
0 upgraded, 0 newly installed, 0 to remove and 12 not upgraded.


In [4]:
!aptitude install mecab libmecab-dev mecab-ipadic-utf8 git make curl xz-utils file -y

mecab is already installed at the requested version (0.996-5)
libmecab-dev is already installed at the requested version (0.996-5)
mecab-ipadic-utf8 is already installed at the requested version (2.7.0-20070801+main-1)
git is already installed at the requested version (1:2.17.1-1ubuntu0.12)
make is already installed at the requested version (4.1-9.1ubuntu1)
curl is already installed at the requested version (7.58.0-2ubuntu3.20)
xz-utils is already installed at the requested version (5.2.2-1.3ubuntu0.1)
file is already installed at the requested version (1:5.32-2ubuntu0.4)
mecab is already installed at the requested version (0.996-5)
libmecab-dev is already installed at the requested version (0.996-5)
mecab-ipadic-utf8 is already installed at the requested version (2.7.0-20070801+main-1)
git is already installed at the requested version (1:2.17.1-1ubuntu0.12)
make is already installed at the requested version (4.1-9.1ubuntu1)
curl is already installed at the requested version (7.58.0-2u

In [5]:
!pip install mecab-python3

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [24]:
import pandas as pd
import torch
import numpy as np

# GPUが使えれば利用する設定
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [7]:
dir = "/content/drive/My Drive/hate-speech-detection-nishika/data/raw/"

train_raw = pd.read_csv(f"{dir}train.csv")
train = train_raw
train.head()

,id,source,text,label
0,80074aa43,news4vip,まともに相手されてない人との関係なんて\nそんな大事にするものか？,0
1,6378fea6b,livejupiter,最近はアヘアヘQSマンやない？ ｲｲ!(・∀・)+1-0(・Ａ・)ｲｸﾅｲ!,0
2,c535f5613,livejupiter,日本人として生まれても無能な低学歴って分かったら日本人の権利剥奪して追放すべきやろ\n甘えるな,1
3,e76638295,livejupiter,よくよく思えば川上は配布にしたらとんでもなく有能だよな\nガチャから引いたら圧倒的歓喜レベルやで,0
4,51e4036bf,newsplus,押井は原作レイプの専門家だから\n原作マンガの真意を誤解させることに関してはプロだが\nそれ...,0


In [8]:
# データの抽出
sentences = train.text.values
labels = train.label.values

In [9]:
# 1. BERT Tokenizerを用いて単語分割・IDへ変換
## Tokenizerの準備
from transformers import BertJapaneseTokenizer

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


Moving 0 files to the new cache system


0it [00:00, ?it/s]

In [10]:
!pip install fugashi

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [11]:
!pip install ipadic

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [12]:
# Load pre-trained tokenizer
tokenizer = BertJapaneseTokenizer.from_pretrained('cl-tohoku/bert-base-japanese-whole-word-masking')

In [13]:
## テスト実行
for i in range(10):
  # 元文章
  print(' Original: ', sentences[i])
  # Tokenizer
  print('Tokenized: ', tokenizer.tokenize(sentences[i]))
  # Token-id
  print('Token IDs: ', tokenizer.convert_tokens_to_ids(tokenizer.tokenize(sentences[i])))
  print("=" * 20)

 Original:  まともに相手されてない人との関係なんて
そんな大事にするものか？
Tokenized:  ['まとも', 'に', '相手', 'さ', 'れ', 'て', 'ない', '人', 'と', 'の', '関係', 'なんて', 'そんな', '大事', 'に', 'する', 'もの', 'か', '?']
Token IDs:  [23135, 7, 1879, 26, 20, 16, 80, 53, 13, 5, 633, 15060, 4799, 15872, 7, 34, 120, 29, 2935]
 Original:  最近はアヘアヘQSマンやない？ ｲｲ!(・∀・)+1-0(・Ａ・)ｲｸﾅｲ!
Tokenized:  ['最近', 'は', 'ア', '##ヘ', '##ア', '##ヘ', 'Q', '##S', 'マン', 'や', 'ない', '?', 'イ', '##イ', '!(', '・', '[UNK]', '・', ')', '##+', '1', '-', '0', '(', '・', 'A', '・', ')', 'イ', '##ク', '##ナイ', '!']
Token IDs:  [5233, 9, 43, 28962, 28483, 28962, 2984, 28583, 1490, 49, 80, 2935, 88, 28478, 21567, 35, 1, 35, 24, 29676, 17, 61, 518, 23, 35, 192, 35, 24, 88, 28488, 4114, 679]
 Original:  日本人として生まれても無能な低学歴って分かったら日本人の権利剥奪して追放すべきやろ
甘えるな
Tokenized:  ['日本人', 'として', '生まれ', 'て', 'も', '無', '##能', 'な', '低', '学歴', 'って', '分かっ', 'たら', '日本人', 'の', '権利', '剥奪', 'し', 'て', '追放', 'す', 'べき', 'やろ', '甘', '##える', 'な']
Token IDs:  [2839, 50, 1115, 16, 28, 348, 28742, 18, 837, 26486, 6172, 8056, 3318,

In [14]:
# 最大単語数の確認
max_len = []
# 1文づつ処理
for sent in sentences:
    # Tokenizeで分割
    token_words = tokenizer.tokenize(sent)
    # 文章数を取得してリストへ格納
    max_len.append(len(token_words))
# 最大の値を確認
print('最大単語数: ', max(max_len))
print('上記の最大単語数にSpecial token（[CLS], [SEP]）の+2をした値が最大単語数')

最大単語数:  91
上記の最大単語数にSpecial token（[CLS], [SEP]）の+2をした値が最大単語数


In [15]:
input_ids = []
attention_masks = []

# 1文づつ処理
for sent in sentences:
    encoded_dict = tokenizer.encode_plus(
                        sent,                      
                        add_special_tokens = True, # Special Tokenの追加
                        max_length = 91,           # 文章の長さを固定（Padding/Trancatinating）
                        pad_to_max_length = True,# PADDINGで埋める
                        return_attention_mask = True,   # Attention maskの作成
                        return_tensors = 'pt',     #  Pytorch tensorsで返す
                   )

    # 単語IDを取得    
    input_ids.append(encoded_dict['input_ids'])

    # Attention　maskの取得
    attention_masks.append(encoded_dict['attention_mask'])

# リストに入ったtensorを縦方向（dim=0）へ結合
input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)

# tenosor型に変換
labels = torch.tensor(labels)

# 確認
print('Original: ', sentences[0])
print('Token IDs:', input_ids[0])

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2308: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


Original:  まともに相手されてない人との関係なんて
そんな大事にするものか？
Token IDs: tensor([    2, 23135,     7,  1879,    26,    20,    16,    80,    53,    13,
            5,   633, 15060,  4799, 15872,     7,    34,   120,    29,  2935,
            3,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0])


In [16]:
from torch.utils.data import TensorDataset, random_split
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

# データセットクラスの作成
dataset = TensorDataset(input_ids, attention_masks, labels)

# 90%地点のIDを取得
train_size = int(0.9 * len(dataset))
val_size = len(dataset) - train_size

# データセットを分割
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

print('訓練データ数：{}'.format(train_size))
print('検証データ数: {} '.format(val_size))

# データローダーの作成
batch_size = 32

# 訓練データローダー
train_dataloader = DataLoader(
            train_dataset,  
            sampler = RandomSampler(train_dataset), # ランダムにデータを取得してバッチ化
            batch_size = batch_size
        )

# 検証データローダー
validation_dataloader = DataLoader(
            val_dataset, 
            sampler = SequentialSampler(val_dataset), # 順番にデータを取得してバッチ化
            batch_size = batch_size
        )

訓練データ数：4730
検証データ数: 526 


In [17]:
from transformers import BertForSequenceClassification, AdamW, BertConfig

# BertForSequenceClassification 学習済みモデルのロード
model = BertForSequenceClassification.from_pretrained(
    "cl-tohoku/bert-base-japanese-whole-word-masking", # 日本語Pre trainedモデルの指定
    num_labels = 2, # ラベル数（今回はBinayなので2、数値を増やせばマルチラベルも対応可）
     output_attentions = False, # アテンションベクトルを出力するか
     output_hidden_states = False, # 隠れ層を出力するか
)

Some weights of the model checkpoint at cl-tohoku/bert-base-japanese-whole-word-masking were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialize

In [18]:
# モデルをGPUへ転送
model.cuda()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(32000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [19]:
# 最適化手法の設定
optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)

# 訓練パートの定義
def train(model):
    model.train() # 訓練モードで実行
    train_loss = 0
    for batch in train_dataloader: # train_dataloaderはword_id, mask, labelを出力する点に注意
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
        optimizer.zero_grad()
        loss = model(b_input_ids, 
                            token_type_ids=None, 
                            attention_mask=b_input_mask, 
                            labels=b_labels).loss # 戻り値とここを修正
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        train_loss += loss.item()
    return train_loss

# テストパートの定義
def validation(model):
    model.eval() # 訓練モードをオフ
    val_loss = 0
    with torch.no_grad(): # 勾配を計算しない
        for batch in validation_dataloader:
            b_input_ids = batch[0].to(device)
            b_input_mask = batch[1].to(device)
            b_labels = batch[2].to(device)
            with torch.no_grad():        
                loss = model(b_input_ids, 
                                    token_type_ids=None, 
                                    attention_mask=b_input_mask,
                                    labels=b_labels).loss # 戻り値とここを修正
            val_loss += loss.item()
    return val_loss

In [20]:
# 学習の実行
max_epoch = 4
train_loss_ = []
test_loss_ = []

for epoch in range(max_epoch):
    train_ = train(model)
    test_ = train(model)
    train_loss_.append(train_)
    test_loss_.append(test_)

In [21]:
# 検証方法の確認（1バッチ分で計算ロジックに確認）

model.eval()# 訓練モードをオフ
for batch in validation_dataloader:
    b_input_ids = batch[0].to(device)
    b_input_mask = batch[1].to(device)
    b_labels = batch[2].to(device)
    with torch.no_grad():   
        # 学習済みモデルによる予測結果をpredsで取得     
        preds = model(b_input_ids, 
                            token_type_ids=None, 
                            attention_mask=b_input_mask)

In [22]:
## 予測結果の確認
print(f'出力:{preds}')

出力:SequenceClassifierOutput(loss=None, logits=tensor([[ 6.1468, -4.6772],
        [ 6.0583, -4.7896],
        [ 5.8995, -4.4553],
        [ 6.1643, -4.9141],
        [ 5.8679, -4.3755],
        [ 5.9769, -4.7143],
        [ 5.6395, -4.1459],
        [ 6.0708, -4.7252],
        [ 5.7757, -4.3442],
        [ 6.1546, -4.7892],
        [ 5.9795, -4.5440],
        [ 6.1972, -4.8766],
        [-3.6924,  2.9827],
        [ 6.2017, -4.8550]], device='cuda:0'), hidden_states=None, attentions=None)


In [25]:
# pd.dataframeへ変換（GPUに乗っているTensorはgpu->cpu->numpy->dataframeと変換）
logits_df = pd.DataFrame(preds[0].cpu().numpy(), columns=['logit_0', 'logit_1'])
## np.argmaxで大き方の値を取得
pred_df = pd.DataFrame(np.argmax(preds[0].cpu().numpy(), axis=1), columns=['pred_label'])
label_df = pd.DataFrame(b_labels.cpu().numpy(), columns=['true_label'])

accuracy_df = pd.concat([logits_df, pred_df, label_df], axis=1)

accuracy_df.head()

,logit_0,logit_1,pred_label,true_label
0,6.146763,-4.677162,0,0
1,6.058300,-4.789567,0,0
2,5.899467,-4.455338,0,0
3,6.164337,-4.914114,0,0
4,5.867900,-4.375484,0,0


In [48]:
accuracy_df.shape

(14, 4)

# 予測

In [53]:
test_raw = pd.read_csv(f"{dir}test.csv")
test = test_raw
test.head()

,id,source,text
0,001026808,news4vip,上でも言ったけどオタクレベルの知識求めてる訳じゃない\nただ囲碁やります！って人が誰1人プロ...
1,00465ac96,livejupiter,たとえば、黒人なんかは、生物学的欠陥はないのに、文化的要因で、悪循環に陥り、実力をつけられず...
2,004674725,livejupiter,そうなんやろなあ色々と勿体ない感じしたわ\n終わり方と黒幕キャラは好きやったで\n\nちなワ...
3,00474460f,news4vip,法的というか自治体ごとにバラバラの条例で定めてるだけだからな\n普通の淫行条例だと「青少年に...
4,004a7525c,newsplus,別のジャーナリストの感想として言われてるので客観的な事実とは言えないけど、\n現地は不測の事...


In [70]:
# データの抽出
sentences_test = test.text.values

input_ids_test = []
attention_masks_test = []

# 1文づつ処理
for sent in sentences_test:
    encoded_dict = tokenizer.encode_plus(
                        sent,                      
                        add_special_tokens = True, # Special Tokenの追加
                        max_length = 91,           # 文章の長さを固定（Padding/Trancatinating）
                        pad_to_max_length = True,# PADDINGで埋める
                        return_attention_mask = True,   # Attention maskの作成
                        return_tensors = 'pt',     #  Pytorch tensorsで返す
                   )

    # 単語IDを取得    
    input_ids_test.append(encoded_dict['input_ids'])

    # Attention　maskの取得
    attention_masks_test.append(encoded_dict['attention_mask'])

print(len(input_ids_test))
print(len(attention_masks_test))

# リストに入ったtensorを縦方向（dim=0）へ結合
input_ids_test = torch.cat(input_ids_test, dim=0)
attention_masks_test = torch.cat(attention_masks_test, dim=0)

# 確認
print('Original: ', sentences_test[0])
print('Token IDs:', input_ids_test[0])

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2308: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


3223
3223
Original:  上でも言ったけどオタクレベルの知識求めてる訳じゃない
ただ囲碁やります！って人が誰1人プロ棋士わかりません、布石も知りませんなら申し訳ないけどお祈りかな
Token IDs: tensor([    2,   109,   962,  3083,    10, 11218, 25678,  2900,     5,  4125,
         2023,  7134,  3218,  4847,    80,   909, 11155,  4710,  2610,   679,
         6172,    53,    14,  3654,    17,    53,   285, 10400, 14428,  6769,
         1058,     6,  3432, 28922,    28,  4534,  6769,  1058,   737,  4482,
        29522,    80, 11218,    73, 30284, 28477,    29,    18,     3,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0])


In [75]:
# データセットクラスの作成
test_dataset = TensorDataset(input_ids_test, attention_masks_test)

# データローダー
test_dataloader = DataLoader(
            test_dataset, 
            sampler = SequentialSampler(test_dataset), # 順番にデータを取得してバッチ化
            batch_size = batch_size
        )

In [72]:
print(batch_size)

32


In [80]:
list_wakaran = []

# testデータを予測
model.eval()# 訓練モードをオフ
for batch in test_dataloader:
  
    b_input_ids = batch[0].to(device)
    b_input_mask = batch[1].to(device)
    with torch.no_grad():   
        # 学習済みモデルによる予測結果をpredsで取得     
        preds = model(b_input_ids, 
                            token_type_ids=None, 
                            attention_mask=b_input_mask)
        
print(preds)
# pd.dataframeへ変換（GPUに乗っているTensorはgpu->cpu->numpy->dataframeと変換）
logits_df = pd.DataFrame(preds[0].cpu().numpy(), columns=['logit_0', 'logit_1'])
## np.argmaxで大き方の値を取得
pred_df = pd.DataFrame(np.argmax(preds[0].cpu().numpy(), axis=1), columns=['pred_label'])

sub_df = pd.concat([logits_df, pred_df], axis=1)

sub_df.head()

SequenceClassifierOutput(loss=None, logits=tensor([[ 6.1020, -4.9324],
        [ 5.9201, -4.2099],
        [ 6.0720, -4.7944],
        [ 5.7130, -4.1942],
        [ 6.1347, -4.7427],
        [ 6.1977, -4.8221],
        [ 6.1566, -4.9264],
        [ 6.0806, -4.7799],
        [ 6.1437, -4.7584],
        [-3.6402,  3.1957],
        [-4.9136,  3.6865],
        [ 6.1652, -4.8344],
        [ 6.1122, -4.7379],
        [ 5.8553, -4.3990],
        [ 6.1630, -4.7428],
        [ 5.8369, -4.6909],
        [ 6.0296, -4.5642],
        [-3.9905,  2.9826],
        [ 6.1240, -4.8357],
        [ 6.0097, -4.4919],
        [ 6.0185, -4.3896],
        [ 6.1251, -4.6287],
        [ 6.1395, -4.8951]], device='cuda:0'), hidden_states=None, attentions=None)


,logit_0,logit_1,pred_label
0,6.101994,-4.932401,0
1,5.920081,-4.209941,0
2,6.072003,-4.794415,0
3,5.712966,-4.194177,0
4,6.134705,-4.742689,0


In [81]:
sub_df.shape

(23, 3)